In [1]:
from pathlib import Path

import pandas as pd

In [ ]:
LOG_PATH = Path("..", "log").with_suffix(".jsonl")

In [4]:
def get_all_files_matching(**kwargs):
    """
    Extract all rows where the kwargs are matched.
    """
    log_files = pd.read_json(LOG_PATH, lines=True)
    for key, value in kwargs.items():
        log_files = log_files[log_files[key] == value]
    return log_files

In [ ]:
files = get_all_files_matching(
    model_name="/public/hf/models/meta-llama/Meta-Llama-3.1-8B-Instruct/",
    split="answerable",
)

In [36]:
Path(files["filename"][0]).name

'2025-05-08T19:25:53.jsonl'

In [39]:
p = Path("..", "runs", f"{Path(files['filename'][0]).name}")
with p.open("r") as f:
    df = pd.read_json(f, lines=True)

In [40]:
df

,question,actions,answer,metadata,messages
0,What was the mean Population ages 0-14 (% of t...,"[{'name': 'get_country_codes_in_region', 'argu...",3.222426e+01,"{'question_template': 'AverageProperty', 'slot...","[{'role': 'system', 'content': 'You are a help..."
1,What was the mean Ores and metals exports (% o...,"[{'name': 'get_country_codes_in_region', 'argu...",4.728116e+00,"{'question_template': 'AverageProperty', 'slot...","[{'role': 'system', 'content': 'You are a help..."
2,"What was the mean External debt stocks, total ...","[{'name': 'get_country_codes_in_region', 'argu...",3.796131e+10,"{'question_template': 'AverageProperty', 'slot...","[{'role': 'system', 'content': 'You are a help..."
3,What was the mean Price level ratio of PPP con...,"[{'name': 'get_country_codes_in_region', 'argu...",3.134495e-01,"{'question_template': 'AverageProperty', 'slot...","[{'role': 'system', 'content': 'You are a help..."
4,"What was the mean Unemployment, youth male (% ...","[{'name': 'get_country_codes_in_region', 'argu...",1.414000e+01,"{'question_template': 'AverageProperty', 'slot...","[{'role': 'system', 'content': 'You are a help..."
...,...,...,...,...,...
95,What was the mean Energy intensity level of pr...,"[{'name': 'get_country_codes_in_region', 'argu...",5.392222e+00,"{'question_template': 'AverageProperty', 'slot...","[{'role': 'system', 'content': 'You are a help..."
96,What was the mean Age dependency ratio (% of w...,"[{'name': 'get_country_codes_in_region', 'argu...",5.954529e+01,"{'question_template': 'AverageProperty', 'slot...","[{'role': 'system', 'content': 'You are a help..."
97,"What was the mean Unemployment, male (% of mal...","[{'name': 'get_country_codes_in_region', 'argu...",2.800364e+00,"{'question_template': 'AverageProperty', 'slot...","[{'role': 'system', 'content': 'You are a help..."
98,What was the mean Access to electricity (% of ...,"[{'name': 'get_country_codes_in_region', 'argu...",1.000000e+02,"{'question_template': 'AverageProperty', 'slot...","[{'role': 'system', 'content': 'You are a help..."


In [19]:
results_dataframes = [pd.read_json(f, lines=True) for f in files["filename"].to_list()]

/tmp/ipykernel_48913/559635882.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  results_dataframes = [pd.read_json(f, lines=True) for f in files["filename"].to_list()]


ValueError: Expected object or value

In [ ]:
row["messages"].to_dict()

{8: [{'role': 'system',
   'content': "You are a helpful assistant tasked with answering questions that require multiple intermediate steps of reasoning to arrive at a final answer.\n\nThe questions involve using World Bank data for various countries and indicators, and you have access to a set of tools that can help you retrieve and process this data.\n\nStep-by-step, use the tools provided to execute that plan. Use the `think` tool to think aloud about the steps you will take to solve the problem. Use the `final_answer` tool to return your final answer.\n\nUse the results of each tool call to inform your next step. Note that passing tool calls as the arguments to other tool calls is not allowed. Instead, execute each tool call separately and use the results to perform subsequent calls.\n\nIf a tool call fails, use the error message to help you debug the issue, re-plan, and try again if possible.\n\nBear in mind that some data may not be available for certain countries, indicators, or

In [ ]:
def get_model_final_answer(row):
    # Get the final answer from the row
    messages = row["messages"].to_dict()[row.index[0]]
    # Get the messages where 'role' = 'tool' and 'name' = 'final_answer'
    final_answer = [
        msg
        for msg in messages
        if msg["role"] == "tool" and msg["name"] == "final_answer"
    ]
    if final_answer:
        # Get the content of the final answer
        final_answer_content = final_answer[0]["content"]
        # Convert the content to a string
        final_answer_str = str(final_answer_content)
        return final_answer_str
    else:
        # If no final answer is found, return None
        return None


def get_gold_final_answer(row):
    # Get the final answer from the actions sequence
    actions = row["actions"].to_dict()[row.index[0]]
    # Get the actions where 'name' = 'final_answer'
    final_answer = [action for action in actions if action["name"] == "final_answer"]
    return final_answer["result"]


def get_model_tool_calls(row):
    # Get the tool calls from the row
    messages = row["messages"].to_dict()[row.index[0]]
    # Get the messages where 'role' = 'tool'
    tool_calls = [
        msg["tool_calls"][0]["function"]
        for msg in messages
        if msg["role"] == "assistant" and "tool_calls" in msg
    ]
    return tool_calls


def get_gold_tool_calls(row):
    # Get the tool calls from the actions sequence
    actions = row["actions"].to_dict()[row.index[0]]
    return actions